In [1]:
import numpy as np
import pandas as pd
from one.api import ONE

from psyfun import io

In [4]:
one = ONE()

In [5]:
# Load recordings from previous query
df_recordings = io.load_recordings()

AttributeError: module 'psyfun.io' has no attribute 'load_recordings'

In [7]:
# Pick an example eid
eids = df_recordings['eid'].unique()
eid = eids[0]

In [8]:
# List video-related datasets
[dataset for dataset in one.list_datasets(eid) if 'Camera' in dataset]

['alf/_ibl_bodyCamera.dlc.pqt',
 'alf/_ibl_bodyCamera.times.npy',
 'alf/_ibl_leftCamera.dlc.pqt',
 'alf/_ibl_leftCamera.times.npy',
 'alf/_ibl_rightCamera.dlc.pqt',
 'alf/_ibl_rightCamera.times.npy',
 'alf/bodyCamera.ROIMotionEnergy.npy',
 'alf/leftCamera.ROIMotionEnergy.npy',
 'alf/rightCamera.ROIMotionEnergy.npy',
 'raw_video_data/_iblrig_bodyCamera.frameData.bin',
 'raw_video_data/_iblrig_bodyCamera.raw.mp4',
 'raw_video_data/_iblrig_leftCamera.frameData.bin',
 'raw_video_data/_iblrig_leftCamera.raw.mp4',
 'raw_video_data/_iblrig_rightCamera.frameData.bin',
 'raw_video_data/_iblrig_rightCamera.raw.mp4']

In [17]:
# List video-related datasets
[dataset for dataset in one.list_datasets(eid) if '' in dataset]

['_ibl_experiment.description.yaml',
 'alf/_ibl_bodyCamera.dlc.pqt',
 'alf/_ibl_bodyCamera.times.npy',
 'alf/_ibl_leftCamera.dlc.pqt',
 'alf/_ibl_leftCamera.times.npy',
 'alf/_ibl_rightCamera.dlc.pqt',
 'alf/_ibl_rightCamera.times.npy',
 'alf/bodyCamera.ROIMotionEnergy.npy',
 'alf/bodyROIMotionEnergy.position.npy',
 'alf/leftCamera.ROIMotionEnergy.npy',
 'alf/leftROIMotionEnergy.position.npy',
 'alf/probe00/electrodeSites.brainLocationIds_ccf_2017.npy',
 'alf/probe00/electrodeSites.localCoordinates.npy',
 'alf/probe00/electrodeSites.mlapdv.npy',
 'alf/probe00/pykilosort/_ibl_log.info_pykilosort.log',
 'alf/probe00/pykilosort/_kilosort_whitening.matrix.npy',
 'alf/probe00/pykilosort/_phy_spikes_subset.channels.npy',
 'alf/probe00/pykilosort/_phy_spikes_subset.spikes.npy',
 'alf/probe00/pykilosort/_phy_spikes_subset.waveforms.npy',
 'alf/probe00/pykilosort/channels.brainLocationIds_ccf_2017.npy',
 'alf/probe00/pykilosort/channels.localCoordinates.npy',
 'alf/probe00/pykilosort/channels.m

In [39]:
import ibllib
ibllib.__file__

'/home/crombie/miniconda3/envs/iblenv/lib/python3.10/site-packages/ibllib/__init__.py'

In [6]:
# Download raw video, return path
fpath_video = one.load_dataset(eid, f'*leftCamera.raw*', collection='raw_video_data')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15.0 [00:10<00:00,  1.38it/s]


In [7]:
# Use io.video to stream video data instead of dodwnloading full videos
import ibllib.io.video as vidio
label = 'body' # 'left', 'right' or 'body'

# Find url of video data to stream
url = vidio.url_from_eid(eid, one=one)[label]
# Load video timestamps
ts = one.load_dataset(eid, f'*{label}Camera.times*', collection='alf')
# Find the frame closest to 1000s into data
frame_n = np.searchsorted(ts, 1000)
# Stream the data
frame = vidio.get_video_frame(url, frame_n)
print(frame.shape)

# Load the first 10 video frames
frames = vidio.get_video_frames_preload(url, range(10))
print(frames.shape)

(512, 640, 3)
(10, 512, 640, 3)0


In [20]:
# Get video metadata
meta = vidio.get_video_meta(url, one=one)
for k, v in meta.items():
    print(f'The video {k} = {v}')

The video length = 1682400
The video fps = 30
The video width = 640
The video height = 512
The video duration = 15:34:40
The video size = 12179980136


In [9]:
# Download frameData
for camera in ['left', 'right', 'body']:
    one.load_dataset(eid, f'_iblrig_{camera}Camera.frameData.bin')

(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_video_data/_iblrig_leftCamera.frameData.bin: 100%|█| 
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_video_data/_iblrig_rightCamera.frameData.bin: 100%|█|
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_video_data/_iblrig_bodyCamera.frameData.bin: 100%|█| 


In [10]:
# Use io.camera module to run video QC
from ibllib.qc.camera import CameraQC


In [23]:
CameraQC(eid)

Init signature: CameraQC(session_path_or_eid, camera, **kwargs)
Docstring:      A class for computing camera QC metrics.
Init docstring:
Compute and view camera QC metrics.

:param session_path_or_eid: A session id or path
:param camera: The camera to run QC on, if None QC is run for all three cameras
:param n_samples: The number of frames to sample for the position and brightness QC
:param stream: If true and local video files not available, the data are streamed from
the remote source.
:param log: A logging.Logger instance, if None the 'ibllib' logger is used
:param one: An ONE instance for fetching and setting the QC on Alyx
File:           ~/miniconda3/envs/iblenv/lib/python3.10/site-packages/ibllib/qc/camera.py
Type:           type
Subclasses:     CameraQCCamlog

In [35]:
for task in ['00', '01']:
    _ = one.load_collection(eid, f'raw_task_data_{task}')

(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_task_data_01/_iblrig_encoderPositions.raw.ssv: 100%|█
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_task_data_01/_iblrig_encoderTrialInfo.raw.ssv: 100%|█
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_task_data_01/_iblrig_RFMapStim.raw.bin: 100%|█| 4.05M
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_task_data_01/_iblrig_syncSquareUpdate.raw.csv: 100%|█
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_task_data_01/_iblrig_encoderEvents.raw.ssv: 0.00B [00:00, ?B/s]


In [38]:
# Run camera QC
camera = 'left'  # 'left', 'right', 'body'
qc = CameraQC(eid, camera=camera, sync_collection='raw_task_data', one=one)
qc.run(download_data=True)

2025-01-24 14:46:14 INFO     camera.py:399  Computing QC outcome for left camera, session d77dd225-f7a9-4d01-996b-82c3ae73d56c


TypeError: CameraQC.load_data() got an unexpected keyword argument 'download_data'

> /home/crombie/miniconda3/envs/iblenv/lib/python3.10/site-packages/ibllib/qc/camera.py(402)run()
    400         namespace = f'video{self.label.capitalize()}'
    401         if all(x is None for x in self.data.values()):
--> 402             self.load_data(**kwargs)
    403         if self.data['frame_samples'] is None or self.data['timestamps'] is None:
    404             return spec.QC.NOT_SET, {}



ipdb>  c


In [37]:
qc.run?

Signature: qc.run(update: bool = False, **kwargs) -> (<class 'str'>, <class 'dict'>)
Docstring:
Run video QC checks and return outcome.

:param update: if True, updates the session QC fields on Alyx
:param download_data: if True, downloads any missing data if required
:param extract_times: if True, re-extracts the camera timestamps from the raw data
:returns: overall outcome as a str, a dict of checks and their outcomes
File:      ~/miniconda3/envs/iblenv/lib/python3.10/site-packages/ibllib/qc/camera.py
Type:      method

In [25]:
from ibllib.io import raw_data_loaders as raw
from ibllib.io.session_params import get_task_collection, read_params, get_sync

In [27]:
read_params(qc.session_path)

In [26]:
get_task_collection?

Signature: get_task_collection(sess_params, task_protocol=None)
Docstring:
Fetch the task collection from an experiment description dict.

Parameters
----------
sess_params : dict
    The loaded experiment.description file.
task_protocol : str, optional
    Return the collection that corresponds to this protocol (returns the first matching
    protocol in the list). If None, all collections are returned.

Returns
-------
str, set, None
    If task_protocol is None, returns the set of collections, otherwise returns the first
    collection that corresponds to the protocol, or None if protocol not present.

Notes
-----
- The order of the set may not be the same as the descriptions tasks order when iterating.
File:      ~/miniconda3/envs/iblenv/lib/python3.10/site-packages/ibllib/io/session_params.py
Type:      function

In [21]:
read_params(qc.session_path)

In [20]:
next(qc.session_path.joinpath('raw_video_data').glob(f'_iblrig_{label}Camera.frameData*.bin'), None)

In [17]:
raw.load_embedded_frame_data(qc.session_path, qc.label, raw=True)

(None, None)

In [ ]:
# Run specific video QC check and display the plots
qc.load_data(download_data=True)
qc.check_position(display=True)  # NB: Not all checks make plots
# qc.run()

In [ ]:
from ibllib.qc.camera import run_all_qc
# Run the QC for all cameras
qcs = run_all_qc(eid)

In [22]:
qc.one.eid2path(eid)

PosixPath('/home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001')

In [26]:
one.mode = 'remote'
one.eid2path(eid, 'remote')

PosixPath('/home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001')

In [28]:
vidio.url_from_eid(eid, one=one)

{'body': 'https://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_video_data/_iblrig_bodyCamera.raw.19681bd0-abfc-442a-98c3-b42b3ed70e42.mp4',
 'left': 'https://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_video_data/_iblrig_leftCamera.raw.2154c006-2d60-45b6-9c26-f724c08a6ce0.mp4',
 'right': 'https://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_video_data/_iblrig_rightCamera.raw.2844879c-138f-483d-a0e8-e033ab552975.mp4'}

In [30]:
urlqc = CameraQC('https://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/', 'left', one=one)

In [31]:
urlqc.run()

2025-01-23 17:22:57 INFO     camera.py:399  Computing QC outcome for left camera, session d77dd225-f7a9-4d01-996b-82c3ae73d56c
2025-01-23 17:22:57 INFO     camera.py:203  Gathering data for QC
2025-01-23 17:22:57 WARNING  raw_data_loaders.py:97   No data loaded: could not find raw data file
2025-01-23 17:22:57 WARNING  raw_data_loaders.py:97   No data loaded: could not find raw data file


TypeError: 'NoneType' object is not iterable

> /home/crombie/miniconda3/envs/iblenv_e/lib/python3.10/site-packages/ibllib/io/raw_data_loaders.py(799)load_bpod_fronts()
    797     BNC1_fronts = np.array([[np.nan, np.nan]])
    798     BNC2_fronts = np.array([[np.nan, np.nan]])
--> 799     for tr in data:
    800         BNC1_fronts = np.append(
    801             BNC1_fronts,



ipdb>  c


In [32]:
urlqc.session_path

PosixPath('https:/ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001')

In [33]:
urlqc.eid

'd77dd225-f7a9-4d01-996b-82c3ae73d56c'

In [34]:
eid

'd77dd225-f7a9-4d01-996b-82c3ae73d56c'